# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab = secret탭에 `OPENAI_API_KEY`등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [5]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕, 차은우! 만나서 반가워. 어떻게 도와줄까?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용


In [6]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 과중한 작업으로 끼니를 챙기지 못하는 노동자들의 고충


In [7]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=1,
        max_completion_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return (response.choices[0].message.content)

title_before = '주말 미친 폭우 예상, 모두들 무사하시길'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길
- 교정 제목: "주말 폭우 예보, 모두 안전하시길 바랍니다"


## 연애코치 ReAct

In [8]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [9]:
print(dating_coach('저 이번 주말에 여자친구와 100일인데. 어떡하죠?'))

1. 상황분석:  
현재 여러분은 여자친구와의 100일 기념일을 맞이하고 있습니다. 이런 기념일은 일반적으로 두 사람의 관계에 있어 특별한 의미를 가지며, 서로에 대한 사랑과 감사를 표현할 좋은 기회입니다. 이 특별한 날을 어떻게 기념할지 고민하는 것은 그 자체로 긍정적인 신호입니다. 여러분은 이 날에 특별함을 더하고 싶어하는 마음으로 관련된 여러 가지 아이디어를 찾아볼 필요가 있습니다.

2. 행동계획:  
- **특별한 선물 준비**: 여자친구가 좋아하는 것을 고려하여 선물을 준비합니다. 예를 들어, 그녀가 좋아하는 책, 액세서리, 또는 개인적이고 의미 있는 아이템을 고려해보세요.
- **로맨틱한 데이트 계획**: 특별한 장소를 예약하는 것이 좋습니다. 그녀가 좋아하는 레스토랑이나 카페, 혹은 야외에서 즐길 수 있는 곳을 고려해보세요.
- **감정 표현**: 100일 동안의 추억을 회상하고 서로의 사랑에 대한 선언이나 편지를 준비해서 감정을 솔직하게 표현하는 것도 좋은 방법입니다.
- **사진 촬영**: 특별한 날의 순간을 기록하기 위해 사진촬영을 제안해보세요. 취향에 맞는 포토스폿을 찾아 사진을 남기는 것도 좋은 추억이 됩니다.

3. 실행:  
- 금요일이나 토요일 오전에 선물을 준비하도록 합니다. 너무 늦기 전에 마음에 드는 것을 미리 구매하고 포장해주세요.
- 데이트 장소와 시간은 미리 예약하고, 쉽게 갈 수 있도록 일정을 조정합니다. 그녀가 좋아하는 음식을 미리 조사하면 도움이 됩니다.
- 편지나 사랑의 메시지를 준비한 후, 데이트 중간이나 마지막에 조용한 순간에 읽어주며 감정을 전하세요. 
- 데이트가 끝난 후 찍은 사진들을 공유하고 서로의 상황을 잊지 않겠다는 메시지를 남기세요.

특별한 날을 맞이하여 신경 써준 만큼, 좋은 추억을 만들 수 있을 것입니다. 여자친구분도 여러분의 정성을 느낄 수 있을 거예요!


In [10]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   - 여자친구가 연락이 잘 되지 않은 것에 대해 감정을 상하게 했다.
   - 이 상황이 발생한 이유는 친구들과의 시간에 집중하다 보니 여자친구에게 소홀해진 것이기 때문에, 그녀는 소외감을 느낄 수 있다.
   - 여자친구의 감정을 냉정하게 이해하고, 진심어린 사과와 함께 그녀의 입장에서 이 문제를 바라보는 것이 중요하다.

2. 행동계획:
   - 우선 그녀에게 진솔한 사과의 메시지를 보내어 마음을 열도록 한다. "어제 연락을 잘 못해서 미안해, 너와의 소중한 시간을 소홀히 했다"와 같은 표현이 좋다.
   - 그 다음 만남을 약속하여 직접 만나서 사과하고, 그녀의 기분과 생각을 들어보는 시간을 갖는다.
   - 향후에 비슷한 상황이 반복되지 않도록, 어떤 노력과 계획을 할 것인지에 대해 함께 이야기하는 것도 좋은 접근이 될 수 있다.

3. 실행:
   - 아래와 같은 메시지를 보내보세요: "안녕, 어제 연락이 잘 안 돼서 미안해. 너와의 관계가 정말 소중해. 우리 함께 이야기 나눌 수 있을까?"
   - 만남 날짜를 정한 후, 약속한 날에 가서 진솔한 대화를 나누고 그녀의 감정을 이해해주는 시간을 갖습니다. 그녀가 이야기할 기회를 충분히 주고, 더욱 신뢰감 있는 관계로 발전할 수 있는 방안을 모색하세요.
   - 마지막으로, 과거 비슷한 상황에 대한 대처 방안을 세워 그 방향으로 노력하는 모습을 보여주는 것도 긍정적인 영향을 미칠 것입니다.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼수 있는 적절한 프롬프팅을 작성하세요.


In [13]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물로 별도록 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [14]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - **재료 목록 및 상태**:
     - 소고기: 신선하고 조리 가능한 상태
     - 양파: 신선
     - 표고버섯: 신선
     - 명란젓: 개봉 후 2주 이내, 신선한 상태
     - 된장: 유통기한 내의 양호한 상태

   - **요리 컨셉 및 아이디어**:
     주어진 재료로 한식을 기반으로 한 요리를 제안할 수 있습니다. '소고기 된장찌개' 혹은 '소고기 볶음'을 만들 수 있습니다. 또한 명란젓을 활용하여 '명란젓 비빔밥' 같은 별미를 만들 수도 있습니다.

2. 실행계획:
   - **목표 및 세부 계획**:
     - 목표: 소고기 된장찌개를 만들기
     - 단계별 계획은 다음과 같습니다:
       1. 소고기와 양파, 표고버섯을 준비하여 잘 썰기
       2. 냄비에 물을 끓이고 된장과 소고기를 넣고 조리
       3. 양파와 표고버섯을 추가해 끓이기
       4. 마무리로 명란젓을 옆에 내어 조화롭게 곁들이기

   - **필요한 추가 자료 및 전략**:
     - 소고기 된장찌개에 필요한 육수나 기본 재료 조리 방법
     - 명란젓의 곁들임 방법

3. 검증 및 추가내용 확인:
   - 레시피의 정확성을 검토해본 결과, 소고기 된장찌개 조리 과정에서 물의 양과 끓이는 시간, 명란젓과의 조화로운 서빙 방법을 명확히 설명할 필요가 있습니다.

4. 최종레시피:
   - **필요한 준비물**:
     - 소고기 200g
     - 양파 1개
     - 표고버섯 3~4개
     - 명란젓 1팩
     - 된장 2큰술
     - 물 4컵
     - 다진 마늘(optional)
     - 대파(optional)

   - **레시피 단계**:
     1. 소고기를 식감에 맞게 썰고, 양파는 채 썰고 표고버섯은 슬라이스합니다.
     2. 냄비에 물 4컵을 붓고 끓입니다.
     3. 물이 끓으면 소고기를 넣고 약 5분간 끓이다가 거품을 제거합니다.
     4. 된장을 숟가락으로 풀어 넣고 잘 저어줍니다.
     5. 양파와 표고버섯을 추가하고 중간 불로 10분간 끓입니다.
     6. 마지막으로 다진 마늘과 대파(선택)를 넣고 2분 더 끓입니다.
     7. 완성된 찌개는 그릇에 담아 명란젓을 곁들여 함께 제공합니다. 

이 레시피를 통해 맛있는 소고기 된장찌개를 즐기실 수 있을 것입니다!

## 면접질문 생성 JSON 출력


In [19]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [20]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
print(job_interview(job_posting))

{
    "hard_skill": [
        {
            "question": "Python 프로그래밍 관련 경험에 대해 설명해 주세요. 어떤 프로젝트에서 활용하셨나요?",
            "answer": "저는 대학 시절부터 Python을 활용하여 데이터 분석 및 머신러닝 프로젝트를 진행해왔습니다. 예를 들어, 제 졸업 프로젝트에서는 데이터 전처리와 분석을 위해 Pandas와 NumPy를 활용했고, Scikit-Learn을 사용하여 머신러닝 모델을 구축했습니다. 이 과정에서 Python의 각종 라이브러리를 활용하여 데이터 시각화도 진행했습니다."
        },
        {
            "question": "Docker와 Kubernetes를 활용한 경험에 대해 말씀해 주세요.",
            "answer": "저는 학습 목적으로 개인 프로젝트를 진행하면서 Docker를 사용하여 애플리케이션을 컨테이너화한 경험이 있습니다. 그 프로젝트에서 컨테이너를 Kubernetes 클러스터에 배포하여 스케일링과 로드밸런싱 기능을 적용해 보았습니다. 이를 통해 컨테이너 오케스트레이션에 대한 기본적인 이해를 쌓을 수 있었습니다."
        },
        {
            "question": "Jenkins 또는 GitLab CI/CD 도구를 활용한 경험에 대해 설명해 주세요.",
            "answer": "저는 GitLab CI를 사용하여 자동화된 배포 파이프라인을 설정한 경험이 있습니다. 이를 통해 코드 변경 시 자동으로 빌드와 테스트가 진행되도록 구성하였고, 성공적인 빌드를 통해 개발 환경에 자동 배포될 수 있도록 하였습니다. 이 과정에서 CI/CD의 중요성과 유용성을 깊이 이해하게 되었습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "팀과의 협업에서 어떤 방식으로 의사소통을 하시는 편인가요